In [2]:
from utils import *

model = load_model()
    
r_seq = db2Rseq()
h_seq = db2Hseq(h_step = 4, pred_step = 6)
t = dt2T(pred_step = 6)
s_attrs, _ = db2S()

# output = model(r_seq.float(), h_seq.float(), t.int(), s_attrs.float())

In [3]:
r_seq.shape, h_seq.shape, t.shape, s_attrs.shape

(torch.Size([722, 12, 1]),
 torch.Size([722, 6, 4, 1]),
 torch.Size([722, 6, 3]),
 torch.Size([722, 16]))

In [5]:
# test for lambda
import time
import datetime

now = time.time()
connection = db_connect()
select_query = "SELECT * FROM occupancy"

with connection.cursor() as cursor:
        cursor.execute(select_query)
        result = cursor.fetchall()
        connection.close()

df = pd.DataFrame(result, columns=[col[0] for col in cursor.description])

print('running time:', time.time() - now)
print('time:', now, 'summation:', df.sum().sum())

running time: 0.08759307861328125
time: 1692321029.1656742 summation: 369411
